I am analysing a playlist which contains almost 300 tracks but I want to reduce the size of it.

1. So, I am using Spotify api to fetch data and do the analysis on it.
2. So, I have copied the HTML of that playlist and now using it I am performaing that analysis to reduce the tracks count of the playlist.

In [19]:
import json
from pathlib import Path

import spotipy as sp
from dotenv import load_dotenv
from spotipy.oauth2 import SpotifyClientCredentials

# Fetch Playlist Tracks

## Connect with Spotify API

In [2]:
load_dotenv()
app = sp.Spotify(auth_manager=SpotifyClientCredentials())
app

In [21]:
def get_all_tracks_from_playlist(app: sp.Spotify, playlist_id: str) -> list[dict]:
    """ Fetch all tracks present in a Spotify playlist. """
    # Fetch the first batch of tracks
    response: dict | None = app.playlist_items(playlist_id)
    if response is None:
        raise ValueError("Playlist response is None.")
    all_tracks: list = response["items"]

    # Iterate through the remaining pages
    while response.get("next"):
        response = app.next(response)
        all_tracks.extend(response["items"])

    return all_tracks

## Get playlist data using `playlist_id`

In [22]:
golden_era = get_all_tracks_from_playlist(app, "6W4xIGHYuLXTbWXEGfmwxw")

playlist_path = Path("data/playlist-items.json")
with playlist_path.open("w") as f:
    json.dump(golden_era, f)

# Data Analysis on Playlist

## Import playlist as `pl.DataFrame`

In [5]:
import polars as pl

In [6]:
pl.Config.set_fmt_str_lengths(40)

polars.config.Config

In [7]:
df = pl.read_json("data/playlist-items.json")
print(df.shape)
df.head(2)

(293, 6)


added_at,added_by,is_local,primary_color,track,video_thumbnail
str,struct[5],bool,null,struct[19],struct[1]
"""2022-10-16T06:34:28Z""","{{""https://open.spotify.com/user/31ps2negrszyvigiiyrdqjbqf5gi""},""https://api.spotify.com/v1/users/31ps2negrszyvigiiyrdqjbqf5gi"",""31ps2negrszyvigiiyrdqjbqf5gi"",""user"",""spotify:user:31ps2negrszyvigiiyrdqjbqf5gi""}",false,null,"{{""compilation"",[{{""https://open.spotify.com/artist/0LyfQWJT6nXafLPZqxe9Of""},""https://api.spotify.com/v1/artists/0LyfQWJT6nXafLPZqxe9Of"",""0LyfQWJT6nXafLPZqxe9Of"",""Various Artists"",""artist"",""spotify:artist:0LyfQWJT6nXafLPZqxe9Of""}],[""IN""],{""https://open.spotify.com/album/5JCx0naviMFsPTfmaBmYbj""},""https://api.spotify.com/v1/albums/5JCx0naviMFsPTfmaBmYbj"",""5JCx0naviMFsPTfmaBmYbj"",[{640,""https://i.scdn.co/image/ab67616d0000b273a1fd486925da9490d70a9d2f"",640}, {300,""https://i.scdn.co/image/ab67616d00001e02a1fd486925da9490d70a9d2f"",300}, {64,""https://i.scdn.co/image/ab67616d00004851a1fd486925da9490d70a9d2f"",64}],""Nikaah"",""1982-09-24"",""day"",8,""album"",""spotify:album:5JCx0naviMFsPTfmaBmYbj""},[{{""https://open.spotify.com/artist/62mCohf6aiF3nryWghwCxM""},""https://api.spotify.com/v1/artists/62mCohf6aiF3nryWghwCxM"",""62mCohf6aiF3nryWghwCxM"",""Ghulam Ali"",""artist"",""spotify:artist:62mCohf6aiF3nryWghwCxM""}],[""IN""],1,468826,false,false,{""INH109633640""},{""https://open.spotify.com/track/7nUAHij7tN3aBFaeEqKhMt""},""https://api.spotify.com/v1/tracks/7nUAHij7tN3aBFaeEqKhMt"",""7nUAHij7tN3aBFaeEqKhMt"",false,""Chupke Chupke Raat Din"",50,""https://p.scdn.co/mp3-preview/b92bbb34b0760ee6ca272ae524b782e240ac0ea9?cid=24a0d2915dd944f1b5303b801bae5ba0"",true,5,""track"",""spotify:track:7nUAHij7tN3aBFaeEqKhMt""}",{null}
"""2023-04-26T15:29:48Z""","{{""https://open.spotify.com/user/31ps2negrszyvigiiyrdqjbqf5gi""},""https://api.spotify.com/v1/users/31ps2negrszyvigiiyrdqjbqf5gi"",""31ps2negrszyvigiiyrdqjbqf5gi"",""user"",""spotify:user:31ps2negrszyvigiiyrdqjbqf5gi""}",false,null,"{{""album"",[{{""https://open.spotify.com/artist/0wcJ6Eio1d7Te1lf3B7bVK""},""https://api.spotify.com/v1/artists/0wcJ6Eio1d7Te1lf3B7bVK"",""0wcJ6Eio1d7Te1lf3B7bVK"",""Kaifi Azmi"",""artist"",""spotify:artist:0wcJ6Eio1d7Te1lf3B7bVK""}],[""IN""],{""https://open.spotify.com/album/04KrFv8qKzc0CjztfX3smv""},""https://api.spotify.com/v1/albums/04KrFv8qKzc0CjztfX3smv"",""04KrFv8qKzc0CjztfX3smv"",[{640,""https://i.scdn.co/image/ab67616d0000b273d9fdb5b496b4963c90111a12"",640}, {300,""https://i.scdn.co/image/ab67616d00001e02d9fdb5b496b4963c90111a12"",300}, {64,""https://i.scdn.co/image/ab67616d00004851d9fdb5b496b4963c90111a12"",64}],""Kaifi Azmi Ki Kalam Se"",""2022-05-02"",""day"",20,""album"",""spotify:album:04KrFv8qKzc0CjztfX3smv""},[{{""https://open.spotify.com/artist/2ijWbN5KykTYiBoVmhzCTU""},""https://api.spotify.com/v1/artists/2ijWbN5KykTYiBoVmhzCTU"",""2ijWbN5KykTYiBoVmhzCTU"",""Jagjit Singh"",""artist"",""spotify:artist:2ijWbN5KykTYiBoVmhzCTU""}],[""IN""],1,302733,false,false,{""INH109632470""},{""https://open.spotify.com/track/52Af7bUm7zfhQ8qgWWEDMF""},""https://api.spotify.com/v1/tracks/52Af7bUm7zfhQ8qgWWEDMF"",""52Af7bUm7zfhQ8qgWWEDMF"",false,""Jhuki Jhuki Si Nazar"",55,""https://p.scdn.co/mp3-preview/2d243270d05dd4e2cbb311b71e3f67f9e6f886a2?cid=24a0d2915dd944f1b5303b801bae5ba0"",true,9,""track"",""spotify:track:52Af7bUm7zfhQ8qgWWEDMF""}",{null}


## Normalize dataframe

In [8]:
data = (
    df.select("added_at", "track")
    .unnest("track")
    .select(
        pl.col("id").alias("track_id"),
        pl.col("name").alias("track_name"),
        pl.col("type").alias("track_type"),
        "artists",
        "album",
        "duration_ms",
        "popularity",
        "explicit",
        "added_at",
        "preview_url",
    )
    .with_columns(
        pl.col("album").struct.field("id").alias("album_id"),
        pl.col("album").struct.field("name").alias("album_name"),
        pl.col("album").struct.field("album_type"),
        pl.col("artists")
        .map_elements(lambda x: x.struct.field("name"))
        .alias("artists_name"),
        pl.col("artists")
        .map_elements(lambda x: x.struct.field("id"))
        .alias("artists_id"),
    )
    .drop("album", "artists")
    .with_columns(
        pl.col("added_at").str.to_datetime(),
    )
)
print(data.shape)
data.head()

(293, 13)


track_id,track_name,track_type,duration_ms,popularity,explicit,added_at,preview_url,album_id,album_name,album_type,artists_name,artists_id
str,str,str,i64,i64,bool,"datetime[μs, UTC]",str,str,str,str,list[str],list[str]
"""7nUAHij7tN3aBFaeEqKhMt""","""Chupke Chupke Raat Din""","""track""",468826,50,false,2022-10-16 06:34:28 UTC,"""https://p.scdn.co/mp3-preview/b92bbb34b…","""5JCx0naviMFsPTfmaBmYbj""","""Nikaah""","""compilation""","[""Ghulam Ali""]","[""62mCohf6aiF3nryWghwCxM""]"
"""52Af7bUm7zfhQ8qgWWEDMF""","""Jhuki Jhuki Si Nazar""","""track""",302733,55,false,2023-04-26 15:29:48 UTC,"""https://p.scdn.co/mp3-preview/2d243270d…","""04KrFv8qKzc0CjztfX3smv""","""Kaifi Azmi Ki Kalam Se""","""album""","[""Jagjit Singh""]","[""2ijWbN5KykTYiBoVmhzCTU""]"
"""0Uju0OCAsVwlNSY0mQlGTb""","""Tum Itna Jo Muskura Rahe Ho""","""track""",321311,60,false,2023-04-26 15:30:19 UTC,"""https://p.scdn.co/mp3-preview/11f5e3096…","""04KrFv8qKzc0CjztfX3smv""","""Kaifi Azmi Ki Kalam Se""","""album""","[""Jagjit Singh""]","[""2ijWbN5KykTYiBoVmhzCTU""]"
"""15nZ2OViXR5mNwmSuZhx03""","""Dil Ke Arman Ansuon Men Bah Gaye""","""track""",262989,37,false,2023-09-22 12:20:29 UTC,"""https://p.scdn.co/mp3-preview/5cb8c0ea4…","""5JCx0naviMFsPTfmaBmYbj""","""Nikaah""","""compilation""","[""Salma Agha""]","[""7liaWSL1uPVjIrnYSFDB6V""]"
"""4Pnqpq21ngybetlCK3upTR""","""Yeh Raat Yeh Chandni""","""track""",296240,45,false,2024-02-22 02:47:51 UTC,"""https://p.scdn.co/mp3-preview/553806b4e…","""2YClj18H0MtjOGWwqII1sh""","""Baarishein, Chai Aur Hemant Kumar""","""album""","[""Hemant Kumar""]","[""02Um2HIOrUdsy3wqPBZwsj""]"


## Analysis Begins

In [56]:
# No. of Uninque tracks, albums, artists
print("No. of unique track_id:   ", data["track_id"].n_unique())
print("No. of unique album_id:   ", data["album_id"].n_unique())
print("No. of unique artists_id: ", data["artists_id"].explode().n_unique())

No. of unique track_id:    290
No. of unique album_id:    198
No. of unique artists_id:  62


In [9]:
data.select("track_id", "track_name").filter(pl.col("track_id").is_duplicated())

track_id,track_name
str,str
"""6xCnMMPkIIhn3QyGJgd5xd""","""Ae Mere Zohra Jabeen"""
"""65Jc0dClAN7bfmhLJ9gJyU""","""Samne Ye Kaun Aya - From ""Jawani Diwani…"
"""0UUEg4ozehOADGnhQ4OOam""","""Chahoonga Main Tujhe"""
"""6xCnMMPkIIhn3QyGJgd5xd""","""Ae Mere Zohra Jabeen"""
"""65Jc0dClAN7bfmhLJ9gJyU""","""Samne Ye Kaun Aya - From ""Jawani Diwani…"
"""0UUEg4ozehOADGnhQ4OOam""","""Chahoonga Main Tujhe"""


In [10]:
# Filter track_name by removing the extra infos
data = data.with_columns(
    pl.col("track_name").str.extract(r"([-\(].+)").alias("track_name_extra"),
    pl.col("track_name")
    .str.replace(r"([-\(].+)", "")
    .str.strip_chars_end()
    .alias("track_name_true"),
)
data.sample(3)

track_id,track_name,track_type,duration_ms,popularity,explicit,added_at,preview_url,album_id,album_name,album_type,artists_name,artists_id,track_name_extra,track_name_true
str,str,str,i64,i64,bool,"datetime[μs, UTC]",str,str,str,str,list[str],list[str],str,str
"""6OFlQp1PQ5b1oKMR7L0Vjx""","""Aane Se Uske Aaye Bahar-Duet""","""track""",88712,19,false,2023-10-02 07:25:12 UTC,"""https://p.scdn.co/mp3-preview/06c127099…","""4yNcphRaHZF70l7J0VDsoL""","""Jeene Ki Raah""","""album""","[""Mohammed Rafi"", ""Bhupinder Singh"", ""Laxmikant–Pyarelal""]","[""0gXDpqwYNDODn7fB0RDN8J"", ""4FMGD43a8aLM0LRKXDSXne"", ""3yS84AjNFqhmuJlIXy7sax""]","""-Duet""","""Aane Se Uske Aaye Bahar"""
"""024NXo94cy9NtPSDqBCmMn""","""Mera Joota Hai Japani""","""track""",268533,43,false,2023-04-19 17:18:58 UTC,"""https://p.scdn.co/mp3-preview/5af330461…","""30AnFi0y2n2KZIHMzYhSWX""","""Mukesh Ki Barfi""","""album""","[""Mukesh""]","[""4etv0ut4ws0GbXBtolzf5e""]",null,"""Mera Joota Hai Japani"""
"""1b5MoUyroLfLEdhJOZfQLP""","""Babuji Dheere Chalna""","""track""",207785,50,false,2024-02-21 18:13:09 UTC,"""https://p.scdn.co/mp3-preview/77dd2e5ea…","""75cPtv4f6SZ7aKUAJMHWXJ""","""Vintage Melody Geeta Dutt""","""album""","[""Geeta Dutt""]","[""0QsbYX8XslSSUcztc6u4pO""]",null,"""Babuji Dheere Chalna"""


In [11]:
# no. of unique track_name
print("No. of unique track_name_true", data["track_name_true"].n_unique())

No. of unique track_name_true 269


In [16]:
# Duplicate songs whoes names are different
data.filter(pl.col("track_name_true").is_duplicated()).select(
    "track_name", "album_name", "duration_ms", "added_at", "artists_name"
).sort("track_name")

track_name,album_name,duration_ms,added_at,artists_name
str,str,i64,"datetime[μs, UTC]",list[str]
"""Aane Se Uske Aaye Bahar""","""Jeene Ki Raah""",247613,2022-08-27 10:36:57 UTC,"[""Mohammed Rafi"", ""Laxmikant–Pyarelal""]"
"""Aane Se Uske Aaye Bahar-Duet""","""Jeene Ki Raah""",88712,2023-10-02 07:25:12 UTC,"[""Mohammed Rafi"", ""Bhupinder Singh"", ""Laxmikant–Pyarelal""]"
"""Ae Mere Zohra Jabeen""","""Waqt""",235333,2023-04-19 17:40:10 UTC,"[""Manna Dey""]"
"""Ae Mere Zohra Jabeen""","""Waqt""",235333,2023-09-30 12:48:24 UTC,"[""Manna Dey""]"
"""Babuji Dheere Chalna""","""Aar Paar""",207785,2023-04-26 15:29:38 UTC,"[""Geeta Dutt""]"
"""Babuji Dheere Chalna""","""Vintage Melody Geeta Dutt""",207785,2024-02-21 18:13:09 UTC,"[""Geeta Dutt""]"
"""Chahoonga Main Tujhe""","""Dosti""",294493,2023-05-15 07:19:53 UTC,"[""Mohammed Rafi"", ""Laxmikant–Pyarelal""]"
"""Chahoonga Main Tujhe""","""Dosti""",294493,2023-09-30 12:50:32 UTC,"[""Mohammed Rafi"", ""Laxmikant–Pyarelal""]"
"""Chanda Ko Dhundne""","""Jeene Ki Raah""",279040,2024-02-21 18:21:53 UTC,"[""Asha Bhosle"", ""Usha Mangeshkar"", … ""Laxmikant–Pyarelal""]"


In [19]:
# No. of tracks present in playlist of artists
(
    data["artists_name"]
    .explode()
    .value_counts()
    .filter(
        pl.col("count").ge(20),
    )
    .sort("count", descending=True)
)

artists_name,count
str,u32
"""Kishore Kumar""",97
"""Lata Mangeshkar""",76
"""Mohammed Rafi""",56
"""R. D. Burman""",49
"""Mukesh""",37
"""Laxmikant–Pyarelal""",31
"""Asha Bhosle""",29
"""S. D. Burman""",16
"""Manna Dey""",13


In [102]:
# Top tracks a/c duration
data.select("track_name", "duration_ms").top_k(15, by="duration_ms")

track_name,duration_ms
str,i64
"""Pag Ghunghroo Baandh""",697616
"""Koi Fariyaad""",541346
"""Intaha Ho Gai Intezar Ki""",529635
"""Piya Tose Naina Laage Re""",509466
"""Ek Hasina Thi Ek Diwana Tha""",472946
"""Chupke Chupke Raat Din""",468826
"""Yunhi Kat Jaayega""",463933
"""Zeehale Muskin - From ""Ghulami""""",446440
"""Ankhiyon Ke Jharokhon Se - Part 1""",445707


In [104]:
# First n tracks added in playlist
(
    data.select("added_at", "track_name")
    .with_columns(
        pl.col("added_at").str.to_datetime(),
    )
    .bottom_k(10, by="added_at")
)

added_at,track_name
"datetime[μs, UTC]",str
2022-08-02 15:54:14 UTC,"""Tera Mujhse - From ""Aa Gale Lag Jaa"""""
2022-08-02 15:54:47 UTC,"""Zindagi Ki Yahi Reet Hai"""
2022-08-02 15:55:03 UTC,"""O Saathi Re - Kishore Kumar"""
2022-08-02 15:55:16 UTC,"""Ek Ajnabee Haseena Se"""
2022-08-02 15:55:54 UTC,"""Dekha Ek Khwab - II"""
2022-08-02 15:56:35 UTC,"""Yunhi Kat Jaayega"""
2022-08-02 16:01:22 UTC,"""Main Pal Do Pal Ka Shair Hoon"""
2022-08-02 16:01:29 UTC,"""Samne Ye Kaun Aya - From ""Jawani Diwani…"
2022-08-02 16:01:35 UTC,"""Maine Poochha Chand Se - From ""Abdullah…"


In [103]:
# Last n tracks added in playlist
(
    data.select("added_at", "track_name")
    .with_columns(
        pl.col("added_at").str.to_datetime(),
    )
    .top_k(10, by="added_at")
)

added_at,track_name
"datetime[μs, UTC]",str
2024-03-17 08:10:33 UTC,"""Tere Naam Ka Diwana - From ""Suraj Aur C…"
2024-03-15 09:09:42 UTC,"""Saathiya Nahin Jana Revival Film - Aya …"
2024-03-13 08:04:12 UTC,"""Solah Baras Ki Bali Umar"""
2024-03-13 06:34:48 UTC,"""Yaara O Yaara - Narendra Chanchal"""
2024-03-11 14:42:04 UTC,"""Mere Dil Mein Aaj Kya Hai"""
2024-03-11 14:40:31 UTC,"""Maine Tere Liye"""
2024-03-09 10:54:21 UTC,"""Tumhin Mere Mandir"""
2024-03-05 05:13:15 UTC,"""Mujhe Tum Yaad Karna"""
2024-03-03 15:35:23 UTC,"""Yeh Reshmi Zulfen"""


In [107]:
# Explicit tracks in playlist
data.filter(pl.col("explicit").eq(True)).select("track_name", "album_name")

track_name,album_name
str,str


In [108]:
data.select("popularity", "track_name").top_k(10, by="popularity")

popularity,track_name
i64,str
66,"""Hoshwalon Ko Khabar Kya"""
64,"""Kya Hua Tera Vada"""
63,"""Ek Ajnabee Haseena Se"""
63,"""Mere Mehboob Qayamat Hogi"""
63,"""Main Pal Do Pal Ka Shair Hoon"""
62,"""Pal Pal Dil Ke Paas - From ""Blackmail"""""
61,"""Gulabi Ankhen - From ""The Train"""""
61,"""Bade Achhe Lagte Hain"""
60,"""Likhe Jo Khat Tujhe"""


In [110]:
data.select("popularity", "track_name").filter(pl.col("popularity").gt(0)).bottom_k(
    10, by="popularity"
)

popularity,track_name
i64,str
11,"""Saathiya Nahin Jana Revival Film - Aya …"
12,"""Chanda Ko Dhundne - Pt. 1"""
12,"""Pardesiyon Se Na Ankhiyan Milana - Male…"
13,"""Jo Hamne Daastaan Apnee"""
13,"""Pukarta Chala Hoon Main - Mere Sanam"""
13,"""Palkon Ke Peechhe Se"""
13,"""Teri Aankhon Ke Siva - Female Vocals"""
14,"""Mera Saaya Saath Hoga"""
14,"""Haule Haule Chalo Mere Sajna"""


In [111]:
data.select("popularity", "track_name").filter(pl.col("popularity").eq(0))

popularity,track_name
i64,str
0,"""Chala Jata Hoon"""
0,"""Hai apna dil to aawara"""
0,"""Kora Kagaz Tha Yeh Man Mera"""
0,"""Bachna Ae Hasinon Lo Main Aa Gaya - Fro…"
0,"""Is Mod Se Jate Hain"""
0,"""Zindagi Ka Safar - From ""Safar"""""
0,"""Ek Chatur Naar Karke Sringar - From ""Pa…"
0,"""Lekar Hum Diwana Dil - Duet Version"""
0,"""Kanchi Re Kanchi Re - From ""Hare Rama H…"


In [112]:
data["album_type"].value_counts()

album_type,count
str,u32
"""album""",226
"""compilation""",25
"""single""",42


In [21]:
data.group_by(
    pl.col("added_at").dt.date().alias('date'),
).len().top_k(10, by="len")

added_at,len
date,u32
2024-02-21,33
2022-08-27,31
2024-01-06,28
2022-08-02,19
2023-04-19,18
2023-11-04,18
2024-02-22,16
2023-04-26,11
2023-09-22,10


In [24]:
data.group_by(
    pl.col("added_at").dt.year().alias("year"),
    pl.col("added_at").dt.month().alias("month"),
).len().top_k(10, by="len")

year,month,len
i32,i8,u32
2022,8,55
2024,2,50
2024,1,35
2023,4,34
2023,9,21
2023,11,21
2023,10,16
2023,12,15
2024,3,10


In [25]:
data.group_by(
    pl.col("added_at").dt.year().alias("year"),
).len().top_k(10, by="len")

year,len
i32,u32
2023,128
2024,95
2022,70
